In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} matplotlib

# Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import matplotlib.ticker as ticker
import pandas as pd
import seaborn as sns
import h5py
from pathlib import Path
import time
from tqdm import tqdm
import csv
import os

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, RepeatedKFold, cross_val_score
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.inspection import permutation_importance

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional
from tensorflow.keras.layers import MaxPooling1D, GlobalAveragePooling1D, AveragePooling1D
from tensorflow.keras.layers import Flatten, Dropout, BatchNormalization
from tensorflow.keras import activations

# import lime
# import lime.lime_tabular

%matplotlib inline
plt.rcParams['font.sans-serif'] = "Helvetica"
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.size'] = 16
plt.rcParams['savefig.facecolor'] = 'white'

# %matplotlib notebook
# %config InlineBackend.figure_format='retina'

In [ ]:
print(f'tf.test.is_built_with_cuda(): {tf.test.is_built_with_cuda()}')

print(f'tf.test.is_gpu_available(): {tf.test.is_gpu_available()}')

if tf.__version__[0] == '1':
    print(f'tf.config.experimental_list_devices(): {tf.config.experimental_list_devices()}')
else:
    print(f'tf.config.list_physical_devices("GPU"): {tf.config.list_physical_devices("GPU")}')
    
print(f'tf.test.gpu_device_name(): {tf.test.gpu_device_name()}')

Save keras model: https://www.tensorflow.org/guide/keras/save_and_serialize


# Define functions

In [ ]:
def print_my_results(results_mae, results_mae_per, results_r2, results_r2_per):
    print('MAE:')
    print('\t%.3f (%.3f) overall' % (np.mean(results_mae), np.std(results_mae)))
    for mae,std,idx in zip(np.mean(results_mae_per, axis=0), np.std(results_mae_per, axis=0), idxs):
        print('\t%.3f (%.3f) %s' % (mae, std, labels[idx]) )
    print('R2:')
    print('\t%.3f (%.3f) overall' %(np.mean(results_r2), np.std(results_r2)))
    for r2,std,idx in zip(np.mean(results_r2_per, axis=0), np.std(results_r2_per, axis=0), idxs):
        print('\t%.3f (%.3f) %s' % (r2, std, labels[idx]) )
        
        
def save_my_results(results_mae):
    my_fname = "MLR_results_mae.csv"
    if not os.path.isfile(my_fname):
        with open(my_fname,"w") as fp:
            pass
    with open(my_fname, "a") as f:
        writer = csv.writer(f)
        writer.writerow(results_mae)
        
        
def drop_features(X, n_segments=3, dropval="zero"):
    X_length = X.shape[1]
    idx_full = np.arange(0,X_length)
    idx_segments = np.array_split(idx_full, n_segments)
    X_repeat = list()
    for idx_seg in idx_segments:
        X_drop = np.copy(X)
        if dropval == "zero":
            X_drop[:,idx_seg] = 0.0
        elif dropval == "mean":
            X_drop[:,idx_seg] = np.mean(X) # mean across both axes of X
        X_repeat.append(X_drop)
    return X_repeat


# Import data

In [ ]:
name = "km-adjfactors_p11_s100_n5000_filtered3"
data_path = Path(f'./data_control_{name}.h5')

with h5py.File(data_path, 'r') as f:
    Datasetnames=f.keys()
    print(*list(Datasetnames), sep = "\n")
    trace = f['trace'][:,:200,:] # select time 0-200
    t = f['time'][...]
    adj_factors = f['adjustment_factors'][...]
    cost_terms = f['cost_terms'][...]
    
if trace.shape[0] != adj_factors.shape[0]:
    print('Number of samples do not match for trace and adj_factors!')

print("Number of traces:", trace.shape[0])
labels = ["g_Kr","g_CaL","lambda_B","g_NaCa","g_K1","J_SERCA_bar","lambda_diff","lambda_RyR","g_bCa","g_Na","g_NaL"]

In [ ]:
# randst = np.random.randint(0,100)
# print(randst)
randst = 22

# k-Nearest Neighbors

## Prep data

In [ ]:
idxs = [0,1,4,9]
idx_targets = np.r_[idxs]

X = np.concatenate((trace[:,:,0],trace[:,:,1]),axis=1)
y = adj_factors[:,idx_targets]

print('X shape:',X.shape)
print('Feature shape:',y.shape)

## Define model

In [ ]:
def get_model_knn(n_neighbors=6):
    model = KNeighborsRegressor(n_neighbors=n_neighbors,
                                weights="uniform", # default = uniform
                                n_jobs=None)
    return model


def evaluate_model_knn(X, y):
    results_mae = list()
    results_mae_per = list()
    results_r2 = list()
    results_r2_per = list()
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=randst)
    for train_ix, test_ix in cv.split(X):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        # define and fit model
        model = get_model_knn(n_neighbors=6)
        model.fit(X_train, y_train)
        # evaluate model on test set
        y_hat = model.predict(X_test)
        r2_per = r2_score(y_test, y_hat, multioutput='raw_values')
        r2 = r2_score(y_test, y_hat)
        mae = mean_absolute_error(y_test, y_hat)
        mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
        print('>%.3f' % mae)
        results_mae.append(mae)
        results_mae_per.append(mae_per)
        results_r2.append(r2)
        results_r2_per.append(r2_per)
    return results_mae, results_mae_per, results_r2, results_r2_per

## Run model with k-fold cross-validation

In [ ]:
start = time.time()
results_mae, results_mae_per, results_r2, results_r2_per = evaluate_model_knn(X, y)
stop = time.time()
print('Time of execution: %f' % (stop-start))

print_my_results(results_mae, results_mae_per, results_r2, results_r2_per)

## Run model once

In [ ]:
saveit = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=randst)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = get_model_knn()
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

mae = mean_absolute_error(y_test, y_hat)
mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
mae_of_mean = np.mean(np.abs(y_train.mean(axis=0) - y_test))
r2 = r2_score(y_test, y_hat)
r2_per = r2_score(y_test,y_hat,multioutput='raw_values')
print("MAE:", mae)
print("R2:", r2)
print("Channel:",[labels[i] for i in idxs])
print("MAE:", mae_per)
print("R2:", r2_per)
# save_my_results(np.append(mae_per, [mae,mae_of_mean]))

if saveit:
    np.save('../results/kNN_predicted_p4.npy',y_hat)
    np.save('../results/kNN_true_p4.npy',y_test)

## GridSearchCV

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = get_model_knn()
param_grid = {"n_neighbors": np.arange(1, 25)}
knn_gscv = GridSearchCV(model, param_grid, cv=5)
knn_gscv.fit(X, y)

print(knn_gscv.best_params_)
n_best = knn_gscv.best_params_["n_neighbors"]

model = get_model_knn(n_neighbors=n_best)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)
mae = mean_absolute_error(y_test, y_hat)
r2 = r2_score(y_test,y_hat,multioutput='raw_values')
print("MAE:",mae)
print("R2:", r2)
print([labels[i] for i in idxs])

# Random Forest

## Prep data

In [ ]:
idxs = [0,1,4,9]
# idxs = [0,1]
idx_targets = np.r_[idxs]

X = np.concatenate((trace[:,:,0],trace[:,:,1]),axis=1)
y = adj_factors[:,idx_targets]
feature_names = [f'feat_V {i}' for i in range(int(X.shape[1]/2))] + [f'feat_Ca {i}' for i in range(int(X.shape[1]/2))]

print('X shape:',X.shape)
print('Feature shape:',y.shape)

## Define model

In [ ]:
def get_model_rfr():
    model = RandomForestRegressor()
    return model

def evaluate_model_rfr(X, y):
    results_mae = list()
    results_mae_per = list()
    results_r2 = list()
    results_r2_per = list()
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=randst)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        # define model
        model = get_model_rfr()
        # fit model
        model.fit(X_train, y_train)
        # evaluate model on test set
        y_hat = model.predict(X_test)
        r2_per = r2_score(y_test,y_hat,multioutput='raw_values')
        r2 = r2_score(y_test,y_hat)
        mae = mean_absolute_error(y_test, y_hat)
        mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
        # store result
        print('>%.3f' % mae)
        results_mae.append(mae)
        results_mae_per.append(mae_per)
        results_r2.append(r2)
        results_r2_per.append(r2_per)
    return results_mae, results_mae_per, results_r2, results_r2_per

## Run model with k-fold cross-validation

In [ ]:
start = time.time()
results_mae, results_mae_per, results_r2, results_r2_per = evaluate_model_rfr(X, y)
stop = time.time()
print('Time of execution: %f' % (np.divide(stop-start, 60))

print_my_results(results_mae, results_mae_per, results_r2, results_r2_per)

## Run model once

In [ ]:
saveit = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=randst)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = get_model_rfr()
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

mae = mean_absolute_error(y_test, y_hat)
mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
mae_of_mean = np.mean(np.abs(y_train.mean(axis=0) - y_test))
r2 = r2_score(y_test, y_hat)
r2_per = r2_score(y_test,y_hat,multioutput='raw_values')
print("MAE:", mae)
print("R2:", r2)
print("Channel:",[labels[i] for i in idxs])
print("MAE:", mae_per)
print("R2:", r2_per)
# save_my_results(np.append(mae_per, [mae,mae_of_mean]))

if saveit:
    np.save('../results/rfr_predicted_p4.npy',y_hat)
    np.save('../results/rfr_true_p4.npy',y_test)

## RF feature importance

In [ ]:
importance = model.feature_importances_
# plt.bar([x for x in range(len(importance))], importance)

rf_importances = pd.Series(importance, index=feature_names)

fig,ax = plt.subplots(1,1,figsize=(14,4))
rf_importances.plot.bar(ax=ax, width=1)
tick_spacing = 10
ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

# Support Vector Regression

## Prep data

In [ ]:
idxs = [0,1,4,9]
idx_targets = np.r_[idxs]

X = np.concatenate((trace[:,:,0],trace[:,:,1]),axis=1)
y = adj_factors[:,idx_targets]
# y = adj_factors[:,:]
feature_names = [f'feat_V {i}' for i in range(int(X.shape[1]/2))] + [f'feat_Ca {i}' for i in range(int(X.shape[1]/2))]

print('X shape:',X.shape)
print('Feature shape:',y.shape)

## Define model

In [ ]:
def get_model_svr():
    svr = SVR(kernel='rbf',
          C=100.0,
          epsilon=0.01,
          gamma=0.001
         )

    model = MultiOutputRegressor(svr)
    return model

def evaluate_model_svr(X, y):
    results_mae = list()
    results_mae_per = list()
    results_r2 = list()
    results_r2_per = list()
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=randst)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        # define model
        model = get_model_svr()
        # fit model
        model.fit(X_train, y_train)
        # evaluate model on test set
        y_hat = model.predict(X_test)
        r2_per = r2_score(y_test,y_hat,multioutput='raw_values')
        r2 = r2_score(y_test,y_hat)
        mae = mean_absolute_error(y_test, y_hat)
        mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
        # store result
        print('>%.3f' % mae)
        results_mae.append(mae)
        results_mae_per.append(mae_per)
        results_r2.append(r2)
        results_r2_per.append(r2_per)
    return results_mae, results_mae_per, results_r2, results_r2_per

## Run model with k-fold cross-validation

In [ ]:
start = time.time()
results_mae, results_mae_per, results_r2, results_r2_per = evaluate_model_svr(X, y)
stop = time.time()
print('Time of execution: %f' % (np.divide(stop-start, 60)))

print_my_results(results_mae, results_mae_per, results_r2, results_r2_per)

## Run model once

In [ ]:
saveit = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=randst)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = get_model_svr()
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

mae = mean_absolute_error(y_test, y_hat)
mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
mae_of_mean = np.mean(np.abs(y_train.mean(axis=0) - y_test))
r2 = r2_score(y_test, y_hat)
r2_per = r2_score(y_test,y_hat,multioutput='raw_values')
print("MAE:", mae)
print("R2:", r2)
print("Channel:",[labels[i] for i in idxs])
print("MAE:", mae_per)
print("R2:", r2_per)
# save_my_results(np.append(mae_per, [mae,mae_of_mean]))

if saveit:
    np.save('../results/svr_predicted_p11.npy',y_hat)
    np.save('../results/svr_true_p11.npy',y_test)
    np.savetxt('../results/svr_target_p11.csv',y_test,delimiter=',',fmt='%.6f')

# Multi-Layer Perceptron

## Prep data

In [ ]:
idxs = [0,1,4,9]
idx_targets = np.r_[idxs]

X = np.concatenate((trace[:,:,0],trace[:,:,1]),axis=1)
y = adj_factors[:,idx_targets]
feature_names = [f'feat_V_{i}' for i in range(int(X.shape[1]/2))] + [f'feat_Ca_{i}' for i in range(int(X.shape[1]/2))]

print('X shape:',X.shape)
print('Feature shape:',y.shape)

## Define model

In [ ]:
def get_model_mlp1(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(500, input_dim=n_inputs, kernel_initializer='he_uniform', activation=activations.swish))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam', metrics="mae")
    return model

def get_model_mlp3(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(500, input_dim=n_inputs, kernel_initializer='he_uniform', activation=activations.swish))
#     model.add(Dropout(0.1))
    model.add(Dense(500, activation=activations.swish))
#     model.add(Dropout(0.1))
    model.add(Dense(500, activation=activations.swish))
#     model.add(Dropout(0.1))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam', metrics="mae")
    return model

def evaluate_model_mlp(X, y):
    results_mae = list()
    results_mae_per = list()
    results_r2 = list()
    results_r2_per = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=randst)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        model = get_model_mlp3(n_inputs, n_outputs) # <<<<< SPECIFY WHICH MLP MODEL
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, verbose=0)
#         model.fit(X_train, y_train, verbose=0, epochs=200) # default epochs=100
        y_hat = model.predict(X_test)
        r2_per = r2_score(y_test,y_hat,multioutput='raw_values')
        r2 = r2_score(y_test,y_hat)
        mae = mean_absolute_error(y_test, y_hat)
        mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
        print('>%.3f' % mae)
        results_mae.append(mae)
        results_mae_per.append(mae_per)
        results_r2.append(r2)
        results_r2_per.append(r2_per)
    return results_mae, results_mae_per, results_r2, results_r2_per

## Run model with k-fold cross-validation

In [ ]:
start = time.time()
results_mae, results_mae_per, results_r2, results_r2_per = evaluate_model_mlp(X, y)
stop = time.time()
print('Time of execution: %f min' % (np.divide(stop-start, 60)))

print_my_results(results_mae, results_mae_per, results_r2, results_r2_per)

## Run model once

In [ ]:
n_inputs, n_outputs = X.shape[1], y.shape[1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=randst)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

keras_spec = 0
if keras_spec:
    model = KerasRegressor(build_fn=lambda: get_model_mlp(n_inputs, n_outputs), epochs=200, verbose=0)
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
else:
    model = get_model_mlp(n_inputs, n_outputs)
    model.fit(X_train, y_train, epochs=200, verbose=0)
#     history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=0)
    y_hat = model.predict(X_test)

mae = mean_absolute_error(y_test, y_hat)
mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
mae_of_mean = np.mean(np.abs(y_train.mean(axis=0) - y_test))
r2 = r2_score(y_test, y_hat)
r2_per = r2_score(y_test,y_hat,multioutput='raw_values')
print("MAE:", mae)
print("R2:", r2)
print("Channel:",[labels[i] for i in idxs])
print("MAE:", mae_per)
print("R2:", r2_per)
# save_my_results(np.append(mae_per, [mae,mae_of_mean]))

# np.save('../results/mlp_predicted_p4.npy',y_hat)
# np.save('../results/mlp_true_p4.npy',y_test)

In [ ]:
plt.plot(history.history['mae'], label='train')
plt.plot(history.history['val_mae'], label='test')
plt.legend()
plt.xlabel('Number of epochs')
plt.ylabel('Loss (MAE)')

# Convolutional Neural Network

## Prep data

In [ ]:
idxs = [0,1,4,9]
# idxs = [0]
idx_targets = np.r_[idxs]

X = np.copy(trace)
y = adj_factors[:,idx_targets]
feature_names = [f'feat_V {i}' for i in range(X.shape[1])] + [f'feat_Ca {i}' for i in range(X.shape[1])]

print('X shape:',X.shape)
print('Feature shape:',y.shape)

## Define model

In [ ]:
def get_model_cnn(n_inputs, n_outputs):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, input_shape=n_inputs))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation=activations.swish))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

def evaluate_model_cnn(X, y):
    results_mae = list()
    results_mae_per = list()
    results_r2 = list()
    results_r2_per = list()
    n_inputs, n_outputs = X.shape[1:], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=randst)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        scalers = {}
        for i in range(X_train.shape[2]):
            scalers[i] = StandardScaler()
            X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i])
        for i in range(X_test.shape[2]):
            X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])
        model = get_model_cnn(n_inputs, n_outputs)
        model.fit(X_train, y_train, verbose=0, epochs=200)
        y_hat = model.predict(X_test)
        r2_per = r2_score(y_test,y_hat,multioutput='raw_values')
        r2 = r2_score(y_test,y_hat)
        mae = mean_absolute_error(y_test, y_hat)
        mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
        print('>%.3f' % mae)
        results_mae.append(mae)
        results_mae_per.append(mae_per)
        results_r2.append(r2)
        results_r2_per.append(r2_per)
    return results_mae, results_mae_per, results_r2, results_r2_per

## Run model with k-fold cross-validation

In [ ]:
start = time.time()
results_mae, results_mae_per, results_r2, results_r2_per = evaluate_model_cnn(X, y)
stop = time.time()
print('Time of execution: %f min' % (np.divide(stop-start, 60)))

print_my_results(results_mae, results_mae_per, results_r2, results_r2_per)

## Run model once

In [ ]:
n_inputs, n_outputs = X.shape[1:], y.shape[1]

# prep data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=randst)
n_batch, n_time, n_channel  = X_train.shape[0], X_train.shape[1], X_train.shape[2]
scalers = {}
for i in range(X_train.shape[2]):
    scalers[i] = StandardScaler()
    X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i])
for i in range(X_test.shape[2]):
    X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

# train and test model
model = get_model_cnn(n_inputs, n_outputs)
model.fit(X_train, y_train, epochs=200, verbose=0)
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, verbose=1)
y_hat = model.predict(X_test)

mae = mean_absolute_error(y_test, y_hat)
mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
mae_of_mean = np.mean(np.abs(y_train.mean(axis=0) - y_test))
r2 = r2_score(y_test, y_hat)
r2_per = r2_score(y_test,y_hat,multioutput='raw_values')
print("MAE:", mae)
print("R2:", r2)
print("Channel:",[labels[i] for i in idxs])
print("MAE:", mae_per)
print("R2:", r2_per)
save_my_results(np.append(mae_per, [mae,mae_of_mean]))


# Fully Convolutional Neural Networks (FCN)

## Prep data

In [ ]:
idxs = [0,1,4,9]
idx_targets = np.r_[idxs]

X = np.copy(trace)
y = adj_factors[:,idx_targets]
feature_names = [f'feat_V {i}' for i in range(X.shape[1])] + [f'feat_Ca {i}' for i in range(X.shape[1])]

print('X shape:',X.shape)
print('Feature shape:',y.shape)

## Define model

In [ ]:
def get_model_fcn(n_inputs, n_outputs):
    x = keras.layers.Input(n_inputs)
    drop_out = Dropout(0.1)(x)
    conv1 = keras.layers.Conv1D(filters=64, kernel_size=8, input_shape=n_inputs, padding='same')(x) # default filter 128
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activations.swish)(conv1)

    drop_out = Dropout(0.1)(conv1)
    conv2 = keras.layers.Conv1D(filters=128, kernel_size=5, padding='same')(conv1) # default filter 256
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation(activations.swish)(conv2)

    drop_out = Dropout(0.1)(conv2)
    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding='same')(conv2) # default filter 128
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation(activations.swish)(conv3)

    full = keras.layers.GlobalAveragePooling1D()(conv3)
#     full = keras.layers.GlobalMaxPooling1D()(conv3)
    out = keras.layers.Dense(n_outputs)(full)
    model = keras.models.Model(inputs=x, outputs=out)

    optimizer = keras.optimizers.Adam()
    model.compile(loss='mae',
                  optimizer=optimizer,
                  metrics=['mae'])
    return model

def get_model_fcn_2(n_inputs, n_outputs):
    model = Sequential()
#     model.add(Dropout(0.1))
    model.add(Conv1D(filters=128, kernel_size=8, input_shape=n_inputs, padding='same'))
    model.add(BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    
#     model.add(Dropout(0.1))
    model.add(Conv1D(filters=256, kernel_size=5, input_shape=n_inputs, padding='same'))
    model.add(BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    
#     model.add(Dropout(0.1))
    model.add(Conv1D(filters=128, kernel_size=3, input_shape=n_inputs, padding='same'))
    model.add(BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    
    model.add(GlobalAveragePooling1D())
#     model.add(MaxPooling1D())
#     model.add(AveragePooling1D())
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model
    
def evaluate_model_fcn(X, y):
    results_mae = list()
    results_mae_per = list()
    results_r2 = list()
    results_r2_per = list()
    n_inputs, n_outputs = X.shape[1:], y.shape[1]
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=randst)
    for train_ix, test_ix in cv.split(X):
        X_train, X_test = X[train_ix,:,:], X[test_ix,:,:]
        y_train, y_test = y[train_ix], y[test_ix]
        nbatch, n_time, n_channel  = X_train.shape[0], X_train.shape[1], X_train.shape[2]
        scalers = {}
        for i in range(X_train.shape[2]):
            scalers[i] = StandardScaler()
            X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i])
        for i in range(X_test.shape[2]):
            X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])
        model = get_model_fcn(n_inputs, n_outputs)
        model.fit(X_train, y_train, verbose=0, epochs=200) # default epochs=100
        y_hat = model.predict(X_test)
        r2_per = r2_score(y_test,y_hat,multioutput='raw_values')
        r2 = r2_score(y_test,y_hat)
        mae = mean_absolute_error(y_test, y_hat)
        mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
        print('>%.3f' % mae)
        results_mae.append(mae)
        results_mae_per.append(mae_per)
        results_r2.append(r2)
        results_r2_per.append(r2_per)
    return results_mae, results_mae_per, results_r2, results_r2_per


## Run model with k-fold cross-validation

In [ ]:
start = time.time()
results_mae, results_mae_per, results_r2, results_r2_per = evaluate_model_fcn(X, y)
stop = time.time()
print('Time of execution: %f' % (np.divide(stop-start, 60)))

print_my_results(results_mae, results_mae_per, results_r2, results_r2_per)

## Run model once

In [ ]:
n_inputs, n_outputs = X.shape[1:], y.shape[1]

# prep data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=randst)
n_batch, n_time, n_channel  = X_train.shape[0], X_train.shape[1], X_train.shape[2]
scalers = {}
for i in range(X_train.shape[2]):
    scalers[i] = StandardScaler()
    X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i])
for i in range(X_test.shape[2]):
    X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

# train and test model
model = get_model_fcn(n_inputs, n_outputs)
model.fit(X_train, y_train, epochs=200, verbose=0)
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, verbose=1)
y_hat = model.predict(X_test)

mae = mean_absolute_error(y_test, y_hat)
mae_per = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
mae_of_mean = np.mean(np.abs(y_train.mean(axis=0) - y_test))
r2 = r2_score(y_test, y_hat)
r2_per = r2_score(y_test,y_hat,multioutput='raw_values')
print("MAE:", mae)
print("R2:", r2)
print("Channel:",[labels[i] for i in idxs])
print("MAE:", mae_per)
print("R2:", r2_per)
save_my_results(np.append(mae_per, [mae,mae_of_mean]))


In [ ]:
plt.plot(history.history['mae'], label='train')
plt.plot(history.history['val_mae'], label='test')
plt.legend()
plt.xlabel('Number of epochs')
plt.ylabel('Loss (MAE)')
plt.savefig('/Users/nick/Desktop/FCN_training_history_nb200.png',dpi=200,bbox_inches='tight')
# plt.title('lrate='+str(lrate), pad=-50)

# Long Short-Term Memory (LSTM)

## Prep data

In [ ]:
idxs = [0,1,4,9,10]
idx_targets = np.r_[idxs]

X = trace
y = adj_factors[:,idx_targets]
feature_names = [f'feat_V {i}' for i in range(X.shape[1])] + [f'feat_Ca {i}' for i in range(X.shape[1])]

scaler = StandardScaler()
X[:,:,0] = scaler.fit_transform(X[:,:,0])
X[:,:,1] = scaler.fit_transform(X[:,:,1])

print('X shape:',X.shape)
print('Feature shape:',y.shape)

## Define model

In [ ]:
def get_model_lstm(n_inputs, n_outputs):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=64, batch_input_shape=n_inputs, return_sequences=True)))
    model.add(LSTM(units=64))
#     model.add(Dropout(0.5))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

def evaluate_model_lstm(X, y):
    results_mae = list()
    results_r2 = list()
    n_inputs, n_outputs = X.shape[1:], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=randst)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model_lstm(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=200) # default epochs=100
        y_hat = model.predict(X_test)
        r2 = r2_score(y_test,y_hat,multioutput='raw_values')
        r2_all = r2_score(y_test,y_hat)
        # evaluate model on test set
        mae = model.evaluate(X_test, y_test, verbose=0)
        # store result
        print('>%.3f' % mae)
        results_mae_all.append(mae)
        results_r2_all.append(r2_all)
        results_r2.append(r2)
    return results_mae_all, results_r2_all, results_r2

## Run model with k-fold cross-validation

In [ ]:
start = time.time()
results_mae_all, results_r2_all, results_r2 = evaluate_model_lstm(X, y)
stop = time.time()
print('Time of execution: %f' % (stop-start))

print_my_results(results_mae_all, results_r2_all, results_r2)

## Run model once

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=randst)
model = get_model_lstm(X.shape, y.shape[1])

nb_epoch = 200
for i in tqdm(range(nb_epoch)):
    model.fit(X_train, y_train, epochs=1, shuffle=False, verbose=0)
    model.reset_states()

# model.fit(X_train, y_train, epochs=200, verbose=0)
y_hat = model.predict(X_test)
r2 = r2_score(y_test,y_hat)
mae = model.evaluate(X_test, y_test)

print(mae)
print(r2)

# Explainable algorithms

Start with a trained model, train dataset, test dataset

## LIME (local interpretable model-agnostic explanations)

### Tabular data (2d array)

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=feature_names, verbose=True, mode='regression')


In [ ]:
i = np.random.randint(0, X_test.shape[0])
exp = explainer.explain_instance(X_test[i], model.predict, num_features=400)

### Multi-channel data (3d array)

In [ ]:
feature_names = [str(i) for i in np.arange(X_train.shape[1])]

explainer = lime.lime_tabular.RecurrentTabularExplainer(X_train, feature_names=feature_names, 
                                                        verbose=True, mode='regression', discretize_continuous=False)

In [ ]:
n_targets = y_train.shape[1]
i = np.random.randint(0,X_test.shape[0])
exp = explainer.explain_instance(X_test[i], model.predict, num_features=400, num_samples=5000)
    
# exp.show_in_notebook(show_table=True)

In [ ]:
explanations = exp.as_list()

# extract feature identifier and sort
import re
list1a, list2a = zip(*explanations)
list1b = [re.split('_|-',a) for a in list1a]
list1c, list1d, list1e = zip(*list1b)

my_list = []
for m in range(len(explanations)):
    row = [int(list1c[m]), int(list1e[m]), list2a[m]]
    my_list.append(row)

my_df = pd.DataFrame(data=my_list,columns=['feature','time','weight'])
my_df["time"] = 200-my_df["time"] # TIME IS REPRESENTED AS PRESENT - TIME (t-21)

# my_df.sort_values(["feature","time"], ascending=["True","True"])
my_df = my_df.sort_values(["feature","time"], ascending=["True","True"]).reset_index(drop=True)
display(my_df)

In [ ]:
fig,axs = plt.subplots(2,1,figsize=(8,8), sharex=True, sharey=True)

axs[0].bar(my_df["time"].iloc[:200],my_df["weight"].iloc[:200])
# axs[0].set_ylim([-0.15,0.15])
axs[1].bar(my_df["time"].iloc[201:],my_df["weight"].iloc[201:])
# axs[1].set_ylim([-0.15,0.15])

## Shapley (SHAP) (keras model only)

https://github.com/slundberg/shap#deep-learning-example-with-deepexplainer-tensorflowkeras-models

https://shap-lrjball.readthedocs.io/en/latest/generated/shap.DeepExplainer.html

In [ ]:
import shap

# load JS visualization code to notebook
shap.initjs()

N = 1000
idx_sample = np.random.randint(0,X.shape[0],N)
X_background = X[idx_sample]
# X_background = shap.maskers.Independent(X[idx_sample,:,:])

# explain the model's predictions using SHAP
explainer = shap.DeepExplainer(model, X_background)
shap_values = explainer.shap_values(X_test)

In [ ]:
# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value[0].numpy(), shap_values[0][0,:,0], X_test[0,:,0], matplotlib=False)

In [ ]:
saveplot = 1

ishap = 4
label = labels[idx]
shap.summary_plot(shap_values[ishap][:,:], X_test[:,:], feature_names=feature_names,
#                   plot_type="bar",
                  max_display=20,
                  show=False
                 )
if saveplot:
    plt.savefig(f'../results/shap_values/swarmplot_sv_mlp_{labels[idxs[ishap]][2:]}.png', dpi=200, bbox_inches='tight')

In [ ]:
saveplot = 1

fig,axs = plt.subplots(2,1,figsize=(8,8), sharex=True, sharey=True)
x_plot = np.arange(0,1000,5)
y1_plot = np.abs(shap_values[ishap][:,:200]).mean(axis=0)
y2_plot = np.abs(shap_values[ishap][:,200:]).mean(axis=0)
axs[0].bar(x_plot,y1_plot,width=5)
axs[0].set_ylabel("Feature importance")
axs[1].bar(x_plot,y2_plot,width=5)
axs[1].set_ylabel("Feature importance")
axs[1].set_xlabel("time (ms)")

if saveplot:
    plt.savefig(f"../results/shap_values/barplot_sv_mlp_{labels[idxs[ishap]][2:]}_both.png", dpi=200, bbox_inches="tight")

## Permutation feature importance (sklearn model only)

https://scikit-learn.org/stable/modules/permutation_importance.html

In [ ]:
scorer = "neg_mean_squared_error"

result = permutation_importance(model, X_test, y_test,
                           n_repeats=5,
                           random_state=0,
                           scoring=scorer,
                           n_jobs=None
                          )

In [ ]:
with open("../results/permutation_importance/permutation_mse_mlp.txt","w+") as f:
    for i in result.importances_mean.argsort()[::-1]:
        if result.importances_mean[i] - 2 * result.importances_std[i] > 0:
            f.write(f"{feature_names[i]:<8} "
                  f"{result.importances_mean[i]:.4f}"
                  f" +/- {result.importances_std[i]:.3f}\n")

In [ ]:
saveit = 1

importances = pd.Series(result.importances_mean, index=feature_names)

fig, axs = plt.subplots(2,1,figsize=(14,10), sharex=False, sharey=True)
importances.iloc[:200].plot.bar(yerr=result.importances_std[:200], ax=axs[0], width=1, rot=45)
importances.iloc[200:].plot.bar(yerr=result.importances_std[200:], ax=axs[1], width=1, rot=45)
# ax.set_title("Feature importances using permutation on full model")
axs[0].set_ylabel("Mean decrease")
axs[1].set_ylabel("Mean decrease")
tick_spacing = 10
axs[0].xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
axs[1].xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
fig.tight_layout()

if saveit:
    plt.savefig("../results/permutation_importance/barplot_mse_mlp_all.png", dpi=200, bbox_inches="tight")

## Feature drop-out

For user-defined segments of the feature vector in a test dataset, set values in segment to zero or mean and calculate resulting score metric (MAE, R2). Compare to score metric with full feature vector.

In [ ]:
# First, list MAE and R2 reference values using complete feature set

n_segments = 6
X_repeat = drop_features(X_test, n_segments, dropval="mean")
r2_out = list()
mae_out = list()
for i,X_rep in enumerate(X_repeat):
    print('Dropout segment %d'%(i+1))
    y_hat = model.predict(X_rep)
    mae_all = mean_absolute_error(y_test, y_hat)
    mae = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
    r2 = r2_score(y_test,y_hat,multioutput='raw_values')
    r2_all = r2_score(y_test,y_hat)
    r2_out.append(np.append(r2,r2_all))
    mae_out.append(np.append(mae,mae_all))
    print([labels[i] for i in idxs])
    print(mae_all)
    print(mae)
    print(r2_all)
    print(r2)
    print('\n')
    
y_hat = model.predict(X_test)
mae_all_0 = mean_absolute_error(y_test, y_hat)
mae_0 = mean_absolute_error(y_test, y_hat, multioutput='raw_values')
print('Reference metrics:')
print(mae_all_0)
print(mae_0)
print(r2_score(y_test,y_hat))
print(r2_score(y_test,y_hat,multioutput='raw_values'))
print('\n')

df_out = pd.DataFrame(data=mae_out,columns=[labels[i] for i in idxs]+["overall"])
df_out.set_index(np.arange(1,n_segments+1),inplace=True)
df_out.loc[0]=np.append(mae_0,mae_all_0)
df_out.sort_index().to_csv(f'../results/feature_drop/feature_drop_mean_MLP_mae_n{n_segments}.csv',float_format='%.4f')

In [ ]:
# Plot change in MAE from reference (no feature dropout)
mae_diff = df_out.loc[1:6].values-df_out.loc[0].values

fig,axs=plt.subplots(2,3, figsize=(12,8), sharex=True, sharey=True)
axs=axs.flatten()
for ch in range(mae_diff.shape[1]):
#     axs[ch].grid()
    xplot = np.arange(1,7)
    axs[ch].bar(xplot,mae_diff[:,ch])
#     axs[ch].set_ylim([mae_diff.min()-0.01,mae_diff.max()+0.01])
    axs[ch].set_xticks(xplot)
    if ch<5:
        axs[ch].set_title(f"$\lambda$_{labels[idxs[ch]][2:]}")
    else:
        axs[ch].set_title("Overall")

# axis labels
for ax in axs.flat:
    ax.set(xlabel='Dropped segment', ylabel='Difference in MAE')
for ax in axs.flat:
    ax.label_outer()
    
plt.savefig("../results/feature_drop/bar_diff_mae_MLP.png",dpi=200,bbox_inches="tight")